A company Manufacturing produces 3 products across 2 factories. Goal: Minimize total production cost while meeting demand constraints.

Factory A: Product1=$10/unit, Product2=$12/unit, Capacity=800hrs
Factory B: Product1=$8/unit, Product2=$11/unit, Capacity=700hrs  
Demand: Product1=900, Product2=700, Product3=500


production_optimization.py


In [ ]:
import pulp
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Decision variables
x11 = pulp.LpVariable('FactoryA_Product1', lowBound=0)
x12 = pulp.LpVariable('FactoryA_Product2', lowBound=0) 
x13 = pulp.LpVariable('FactoryA_Product3', lowBound=0)
x21 = pulp.LpVariable('FactoryB_Product1', lowBound=0)
x22 = pulp.LpVariable('FactoryB_Product2', lowBound=0)
x23 = pulp.LpVariable('FactoryB_Product3', lowBound=0)

# Problem formulation
model = pulp.LpProblem("Production_Optimization", pulp.LpMinimize)

# Objective function - minimize total cost
model += (10*x11 + 12*x12 + 15*x13 +  # Factory A
          8*x21 + 11*x22 + 14*x23),    # Factory B

# Constraints
model += x11 + x21 >= 900, "Product1_Demand"
model += x12 + x22 >= 700, "Product2_Demand"
model += x13 + x23 >= 500, "Product3_Demand"
model += 2*x11 + 3*x12 + 4*x13 <= 800, "FactoryA_Capacity"
model += 1.5*x21 + 2.5*x22 + 3.5*x23 <= 700, "FactoryB_Capacity"

# Solve model
model.solve(pulp.PULP_CBC_CMD(msg=0))

# Extract results
results_data = {
    'Variable': ['FactoryA_P1', 'FactoryA_P2', 'FactoryA_P3', 
                'FactoryB_P1', 'FactoryB_P2', 'FactoryB_P3'],
    'Quantity': [x11.varValue, x12.varValue, x13.varValue,
                x21.varValue, x22.varValue, x23.varValue],
    'UnitCost': [10, 12, 15, 8, 11, 14],
    'TotalCost': [10*x11.varValue or 0, 12*x12.varValue or 0, 15*x13.varValue or 0,
                 8*x21.varValue or 0, 11*x22.varValue or 0, 14*x23.varValue or 0]
}

df = pd.DataFrame(results_data)
total_cost = pulp.value(model.objective)

print("Optimal Production Plan:")
print(df.round(1))
print(f"\nTotal Minimum Cost: ${total_cost:,.0f}")

# Visualization 1: Production allocation
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
ax1.pie([x11.varValue or 0, x12.varValue or 0, x13.varValue or 0], 
        labels=['P1', 'P2', 'P3'], autopct='%1.1f%%')
ax1.set_title('Factory A')

ax2.pie([x21.varValue or 0, x22.varValue or 0, x23.varValue or 0], 
        labels=['P1', 'P2', 'P3'], autopct='%1.1f%%')
ax2.set_title('Factory B')
plt.savefig('factory_production.png', dpi=300, bbox_inches='tight')
plt.close()

# Visualization 2: Cost breakdown
plt.figure(figsize=(12, 6))
plt.bar(df['Variable'], df['TotalCost'], color=['#FF9999', '#66B2FF', '#99FF99', 
                                               '#FFCC99', '#FF99CC', '#99FFFF'])
plt.title('Production Cost Breakdown by Product and Factory')
plt.ylabel('Cost ($)')
plt.xticks(rotation=45)
plt.savefig('cost_breakdown.png', dpi=300, bbox_inches='tight')
plt.close()

print("Files generated:")
print("- factory_production.png")
print("- cost_breakdown.png")


ModuleNotFoundError: No module named 'pulp'